In [1]:
import pandas as pd

In [2]:
ground = pd.read_csv('BRITS/air_raw/pm25_ground.txt')
missing = pd.read_csv('BRITS/air_raw/pm25_missing.txt')

In [18]:
ground

,datetime,001001,001002,001003,001004,001005,001006,001007,001008,001009,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,2014/05/01 01:00:00,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,87.0,...,101.0,84.0,117.0,NaN,97.0,87.0,74.0,94.0,112.0,109.0
1,2014/05/01 02:00:00,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,103.0,...,100.0,77.0,109.0,78.0,97.0,84.0,84.0,101.0,123.0,114.0
2,2014/05/01 03:00:00,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,112.0,...,103.0,90.0,105.0,77.0,103.0,83.0,100.0,112.0,143.0,126.0
3,2014/05/01 04:00:00,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,117.0,...,110.0,94.0,105.0,90.0,107.0,88.0,103.0,120.0,138.0,130.0
4,2014/05/01 05:00:00,119.0,109.0,144.0,129.0,115.0,124.0,130.0,116.0,124.0,...,105.0,80.0,104.0,83.0,111.0,85.0,108.0,125.0,145.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2015/04/30 19:00:00,86.0,72.0,70.0,72.0,73.0,63.0,65.0,69.0,74.0,...,130.0,136.0,79.0,69.0,74.0,125.0,94.0,67.0,64.0,79.0
8755,2015/04/30 20:00:00,87.0,73.0,73.0,76.0,83.0,63.0,61.0,62.0,62.0,...,133.0,124.0,64.0,67.0,75.0,124.0,128.0,103.0,69.0,77.0
8756,2015/04/30 21:00:00,80.0,74.0,80.0,70.0,84.0,69.0,72.0,75.0,77.0,...,117.0,123.0,72.0,70.0,82.0,125.0,139.0,166.0,78.0,94.0
8757,2015/04/30 22:00:00,NaN,NaN,87.0,NaN,NaN,76.0,79.0,NaN,82.0,...,178.0,NaN,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
missing

,datetime,001001,001002,001003,001004,001005,001006,001007,001008,001009,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,2014/05/01 01:00:00,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,87.0,...,NaN,84.0,117.0,NaN,97.0,87.0,74.0,94.0,NaN,NaN
1,2014/05/01 02:00:00,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,103.0,...,NaN,77.0,109.0,NaN,97.0,84.0,84.0,101.0,NaN,NaN
2,2014/05/01 03:00:00,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,112.0,...,NaN,90.0,105.0,NaN,103.0,83.0,100.0,112.0,NaN,NaN
3,2014/05/01 04:00:00,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,117.0,...,NaN,94.0,105.0,NaN,107.0,88.0,103.0,120.0,NaN,NaN
4,2014/05/01 05:00:00,NaN,109.0,144.0,129.0,115.0,124.0,130.0,116.0,124.0,...,NaN,80.0,104.0,NaN,111.0,85.0,108.0,125.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2015/04/30 19:00:00,86.0,72.0,70.0,72.0,73.0,63.0,65.0,69.0,74.0,...,NaN,NaN,79.0,69.0,74.0,125.0,94.0,67.0,64.0,NaN
8755,2015/04/30 20:00:00,87.0,73.0,73.0,76.0,83.0,63.0,61.0,62.0,62.0,...,NaN,NaN,64.0,67.0,75.0,124.0,128.0,103.0,69.0,NaN
8756,2015/04/30 21:00:00,80.0,74.0,80.0,70.0,84.0,69.0,72.0,75.0,77.0,...,NaN,NaN,72.0,70.0,82.0,125.0,139.0,166.0,78.0,NaN
8757,2015/04/30 22:00:00,NaN,NaN,87.0,NaN,NaN,76.0,79.0,NaN,82.0,...,NaN,NaN,NaN,74.0,NaN,NaN,NaN,NaN,NaN,NaN
